In [141]:
#import Dependencies
import pandas as pd
import os


In [142]:
#create a varialbe for the file
file = "purchase_data.csv"

In [143]:
#import and view the first 5 lines of the data file
purchase_data_df = pd.read_csv(file)

In [144]:
#get player count
#break to a new df with only the data we need
player_count_df = purchase_data_df.copy()["SN"]
#count unique Screen Names
player_count = player_count_df.nunique()


In [145]:
#Purchasing Analysis(Total)
#Number of Unique Items
number_items_df = purchase_data_df.copy()["Item Name"]
number_items = number_items_df.nunique()

#Average Purchase Price
average_purcahse_df = purchase_data_df.copy()["Price"]
average_purchase = average_purcahse_df.mean()

#Total Number of Purchases

num_purchases_df = purchase_data_df.copy()["Purchase ID"]
num_purchases = num_purchases_df.count()

#Total Revenue
rev_df = purchase_data_df.copy()["Price"]
rev = rev_df.sum()

average_purchase = "${:,.2f}".format(average_purchase)
rev = "${:,.2f}".format(rev)
purchasing_summary = pd.DataFrame({
    "Number of Unique Itesm":[number_items], "Average Price": [average_purchase], "Number of Purchases": num_purchases, "Total Revenus": [rev]
})
purchasing_summary

,Number of Unique Itesm,Average Price,Number of Purchases,Total Revenus
0,179,$3.05,780,"$2,379.77"


In [146]:
#Gender Demographics

grouped_on_SN = purchase_data_df.copy().drop_duplicates(subset=['SN'])

#Percentage and Count of Male Players
male_count = grouped_on_SN.loc[purchase_data_df.copy()["Gender"] == 'Male',:].count()
male_count = male_count["Gender"]
male_percent = male_count/player_count
male_percent = "{:.2%}".format(male_percent)

#Percentage and Count of Female Players
female_count = grouped_on_SN.loc[purchase_data_df.copy()["Gender"] == 'Female',:].count()
female_count = female_count["Gender"]
female_percent = female_count/player_count
female_percent = "{:.2%}".format(female_percent)

#Percentage and Count of Other / Non-Disclosed
other_count = grouped_on_SN.loc[purchase_data_df.copy()["Gender"] == 'Other / Non-Disclosed',:].count()
other_count = other_count["Gender"]
other_percent = other_count/player_count
other_percent = "{:.2%}".format(other_percent)

gender_demo_summary = pd.DataFrame({
    "Gender":["Male", "Female", "Other / Non-Disclosed"],
    "Total Count": [male_count, female_count, other_count],
    "Percentage of Players": [male_percent, female_percent, other_percent]
})

gender_demo_summary

,Gender,Total Count,Percentage of Players
0,Male,484,84.03%
1,Female,81,14.06%
2,Other / Non-Disclosed,11,1.91%


In [147]:
#Purchasing Analysis (Gender)
#The below each broken by gender
gender_grouped_df = purchase_data_df.copy().groupby(["Gender"])
#Purchase Count
purchase_by_gender = gender_grouped_df["Purchase ID"].count()

#Average Purchase Price
avg_by_gender = gender_grouped_df["Price"].mean()

#Total Purchase Value
total_by_gender = gender_grouped_df["Price"].sum()

#Average Purchase Total per Person by Gender


#Summary DF + Formating
gender_purchase_summary = pd.DataFrame({
    "Number of Purchases": purchase_by_gender,
    "Average Purchase": avg_by_gender, 
    "Total Purchase": total_by_gender
})
gender_purchase_summary = gender_purchase_summary.merge(gender_demo_summary,how='left', on="Gender")
gender_purchase_summary["Avg Total Purchase Per Person"] = gender_purchase_summary["Total Purchase"]/gender_purchase_summary["Total Count"]

gender_purchase_summary = gender_purchase_summary[["Gender", "Number of Purchases", "Average Purchase", "Total Purchase","Avg Total Purchase Per Person" ]]
gender_purchase_summary["Average Purchase"] = gender_purchase_summary["Average Purchase"].map("${:.2f}".format)
gender_purchase_summary["Total Purchase"] = gender_purchase_summary["Total Purchase"].map("${:.2f}".format)
gender_purchase_summary["Avg Total Purchase Per Person"] = gender_purchase_summary["Avg Total Purchase Per Person"].map("${:.2f}".format)
gender_purchase_summary

,Gender,Number of Purchases,Average Purchase,Total Purchase,Avg Total Purchase Per Person
0,Female,113,$3.20,$361.94,$4.47
1,Male,652,$3.02,$1967.64,$4.07
2,Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [148]:
#Age Demographics 
#The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)
age_bins = [0,9, 14,19, 24, 29,34, 39, 100]
age_labels = ["Under 10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "Over 40"]

age_summary_1 = purchase_data_df.copy().drop_duplicates(subset=['SN'])
age_summary_1["Age Groups"] = pd.cut(age_summary_1["Age"], age_bins, labels=age_labels)

# Number of players by age
age_summary_1 = age_summary_1.groupby("Age Groups").count()

age_summary_1 = age_summary_1.rename(columns={"Age":"Total Count"})

#Percentage of Players by age
age_summary_1["Number of Players"] = player_count
age_summary_1["Percentage of Players"] = (age_summary_1["Total Count"]/age_summary_1["Number of Players"])*100
age_summary_1["Percentage of Players"] = age_summary_1["Percentage of Players"].map("{:.2f}%".format)
age_summary_1 = age_summary_1[["Total Count" ,"Percentage of Players"]]
age_summary_1

,Total Count,Percentage of Players
Age Groups,,
Under 10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
Over 40,12,2.08%


In [149]:
#Purchasing Analysis(Age)
age_count = purchase_data_df.copy()
age_count["Age Groups"] = pd.cut(age_count["Age"], age_bins, labels=age_labels)
age_count = age_count[['Price', "Age Groups"]]
#count of Players by age
age_count = age_count.groupby("Age Groups").count()
age_count = age_count.rename(columns={"Price":"Purchase Count"})
#Average Purchase Price by age
age_avg= purchase_data_df.copy()
age_avg["Age Groups"] = pd.cut(age_avg["Age"], age_bins, labels=age_labels)
age_avg = age_avg.groupby("Age Groups").mean()
age_avg= age_avg.rename(columns={"Price":"Average Purchase Price"})

#Total Purchase Value by age
age_total = purchase_data_df.copy()
age_total["Age Groups"] = pd.cut(age_total["Age"], age_bins, labels=age_labels)
age_total = age_total.groupby("Age Groups").sum()
age_total= age_total.rename(columns={"Price":"Total Purchase Value"})

#merge to new df
age_summary_2 = pd.merge(age_count, age_avg, how='left', on="Age Groups")
age_summary_2 = age_summary_2[["Purchase Count", "Average Purchase Price"]]
age_summary_2 = age_summary_2.merge(age_total,how='left', on="Age Groups")
age_summary_2 = age_summary_2[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]



#Average Purchase Total per Person by Age Group by age
age_summary_2 = age_summary_2.merge(age_summary_1.copy(), how='left', on="Age Groups")
age_summary_2["Avg Total Purchase Per Person"] = age_summary_2["Total Purchase Value"]/age_summary_2["Total Count"]
age_summary_2 = age_summary_2[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase Per Person"]]

#formatting
age_summary_2["Average Purchase Price"]= age_summary_2["Average Purchase Price"].map("${:.2f}".format)
age_summary_2["Total Purchase Value"]= age_summary_2["Total Purchase Value"].map("${:.2f}".format)

age_summary_2["Avg Total Purchase Per Person"] = age_summary_2["Avg Total Purchase Per Person"].map("${:.2f}".format)



age_summary_2


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Age Groups,,,,
Under 10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
Over 40,13,$2.94,$38.24,$3.19


In [150]:

#Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
top_spend = purchase_data_df.copy()
top_spend_grouped = top_spend.groupby('SN')
#SN & Purchase Count
top_spend_summary = top_spend_grouped.count().sort_values(by="Purchase ID", ascending=False)
top_spend_summary = top_spend_summary[["Purchase ID"]]
#Average Purchase Price
top_spend_avg = top_spend_grouped.mean()
top_spend_avg = top_spend_avg[["Price"]].rename(columns={"Price": "Average Purchase Price"})

top_spend_summary = top_spend_summary.merge(top_spend_avg, on="SN", how="left")

#Total Purchase Value
top_spend_total = top_spend_grouped.sum()
top_spend_total = top_spend_total[["Price"]].rename(columns={"Price":'Total Purchase Value'})

top_spend_summary = top_spend_summary.merge(top_spend_total, on="SN", how="left")

#Summary and Formatting
top_spend_summary = top_spend_summary.head(5)
top_spend_summary["Average Purchase Price"] = top_spend_summary["Average Purchase Price"].map("${:.2f}".format)
top_spend_summary["Total Purchase Value"] = top_spend_summary["Total Purchase Value"].map("${:.2f}".format)

top_spend_summary






,Purchase ID,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Iral74,4,$3.40,$13.62
Idastidru52,4,$3.86,$15.45
Asur53,3,$2.48,$7.44
Inguron55,3,$3.70,$11.11


In [160]:
#Identify the 5 most popular items by purchase count, then list (in a table):
most_pop = purchase_data_df.copy()
most_pop_grouped = most_pop.groupby("Item Name")

#Item ID
#Item Name
#Purchase Count
most_pop_summary = most_pop_grouped.count()
#Total Purchase Value
most_pop_price = most_pop_grouped.sum()

#Item Price
most_pop_summary = most_pop_summary.merge(most_pop_price, on="Item Name", how="left")
most_pop_summary = most_pop_summary[["SN", "Price_y"]]
most_pop_summary["Item Price"] = most_pop_summary["Price_y"]/most_pop_summary["SN"]

#Rearrange and Formating
most_pop_summary = most_pop_summary[["SN","Item Price", "Price_y"]]
most_pop_summary = most_pop_summary.rename(columns={"SN": "Purchase Count", "Price_y":"Total Purchase Value"} )

most_pop_summary_count = most_pop_summary.sort_values(by="Purchase Count", ascending=False)
most_pop_summary_count["Item Price"] = most_pop_summary["Item Price"].map("${:.2f}".format)
most_pop_summary_count["Total Purchase Value"] = most_pop_summary["Total Purchase Value"].map("${:.2f}".format) 
most_pop_summary_count = most_pop_summary_count.head(6)
most_pop_summary_count


,Purchase Count,Item Price,Total Purchase Value
Item Name,,,
Final Critic,13,$4.61,$59.99
"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
Persuasion,9,$3.22,$28.99
Nirvana,9,$4.90,$44.10
"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
Fiery Glass Crusader,9,$4.58,$41.22


In [166]:
#Identify the 5 most profitable items by total purchase value, then list (in a table):
#Item ID
#Item Name
#Purchase Count
#Item Price
#Total Purchase Value
most_pop_summary_profit = most_pop_summary.sort_values(by="Total Purchase Value", ascending=False)
most_pop_summary_profit = most_pop_summary_profit.head(5)
most_pop_summary_profit["Item Price"] = most_pop_summary_profit["Item Price"].map("${:.2f}".format)
most_pop_summary_profit["Total Purchase Value"] = most_pop_summary_profit["Total Purchase Value"].map("${:.2f}".format) 
most_pop_summary_profit

,Purchase Count,Item Price,Total Purchase Value
Item Name,,,
Final Critic,13,$4.61,$59.99
"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
Nirvana,9,$4.90,$44.10
Fiery Glass Crusader,9,$4.58,$41.22
Singed Scalpel,8,$4.35,$34.80
